##### What is it?

Both `ThreadPoolExecutor` and ProcessPoolExecutor` are part of Python's `concurrent.futures` module -- a high-level API designed to make concurrent programming easier and cleaner.

- `ThreadPoolExecutor` uses multiple thread within the same process.

- `ProcessPoolExecutor` uses multiple processes, each running on seperate CPU core.

They provide the same interface for submitting tasks (`submit()`, `map()`, etc.), but they behave differently under the hood.

---

##### Why use it?

These executors simplify parellel and concurrent programming.

- They handle thread/process management automatically.

- They let you run multiple tasks asynchronously or in parallel with minimal setup.

- You can switch between thread-based or process-based parallelism just by changing one line of code -- ideal for testing performance.

---

##### How it works?

- You create an executor (thread or process pool).

- You submit tasks (functions) to it using:

    - `submit()` → returns a Future object representing a running computation.

    - `map()` → runs a function on a list of inputs and returns results in order.

- The executor handles scheduling, execution, and collecting results.

---

##### Syntax

```Python
from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor

with ThreadPoolExecutor(max_workers=4) as executor:
    results = executor.map(func, data)

with ProcessPoolExecutor(max_workers=4) as executor:
    results = executor.map(func, data)
```

Both share the same methods:

- `submit(fn, *args, **kwargs)` -> submit one function.

- `map(fn, iterable)` -> Apply a function to all items.

- `shutdown()` -> Clean up resources.

---

##### Parameters

| Parameter       | Description                                                                                                                                        |
| --------------- | -------------------------------------------------------------------------------------------------------------------------------------------------- |
| **max_workers** | Number of threads or processes in the pool. If `None`, it defaults to number of processors × 5 for threads, or number of processors for processes. |
| **initializer** | A callable executed before each worker starts (only for `ProcessPoolExecutor`).                                                                    |
| **initargs**    | Arguments passed to the initializer.                                                                                                               |
---

##### Illustration Idea

Example 1 -- using ThreadPoolExecutor

In [1]:
from concurrent.futures import ThreadPoolExecutor
import time

def download_file(name):
    print(f'Downloading {name}....')
    time.sleep(2)
    print(f'{name} downloaded.')
    return name

files = ['file1', 'file2', 'file3']

with ThreadPoolExecutor(max_workers=3) as executor:
    results = executor.map(download_file, files)

print('All download complete: ', list(results))

file1 downloaded.
file2 downloaded.
file3 downloaded.
All download complete:  ['file1', 'file2', 'file3']


Explanation:

- All downloads run concurrently using threads.

- Ideal for I/O bound tasks (like network or file I/O).


Example 2 -- Using ProcessPoolExecutor

In [ ]:
from concurrent.futures import ProcessPoolExecutor
import os
import time

def square(n):
    try:
        print(f"Process {os.getpid()} computing square of {n}")
        time.sleep(1)
        return n * n
    except Exception as e:
        print(f"Error in process {os.getpid()}: {e}")

if __name__ == "__main__":
    numbers = [1, 2, 3, 4, 5]

    with ProcessPoolExecutor(max_workers=3) as executor:
        results = executor.map(square, numbers)

    print("Squares:", list(results))


Output:

Process 18120 computing square of 1

Process 6684 computing square of 2

Process 20700 computing square of 3

Process 18120 computing square of 4

Process 6684 computing square of 5

Squares: [1, 4, 9, 16, 25]

Explanation:
 
- Each Process runs on different CPU core.

- Ideal for CPU-bound tasks like computation or data processing.

- Achieves true parallelism since each process has its own Python interpretor.

Example 3: Mixing them for comparison

In [ ]:
import time
from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor

def task(n):
    print(f'Working on {n}')
    time.sleep(1)
    return n * n

data = range(5)

start = time.time()
with ThreadPoolExecutor() as executor:
    list(executor.map(task, data))
print(f'Thread time : {time.time() - start}')

if __name__ == "__main__":
    start = time.time()
    with ProcessPoolExecutor() as executor:
        list(executor.map(task, data))
    print(f'Process time: {time.time() - start}')

Output:

Process 7592 computing square of 1

Process 8188 computing square of 2

Process 7620 computing square of 3

Process 7592 computing square of 4

Process 8188 computing square of 5

Squares: [1, 4, 9, 16, 25]

Observation:
- Threads are faster for tasks waiting on I/) (like downloading, reading files).

- Processes are faster for tasks that use CPU heavily.

---

##### Key Points

| Feature                       | ThreadPoolExecutor     | ProcessPoolExecutor            |
| ----------------------------- | ---------------------- | ------------------------------ |
| Concurrency Type              | Threads (same process) | Processes (separate memory)    |
| Ideal for                     | I/O-bound tasks        | CPU-bound tasks                |
| GIL (Global Interpreter Lock) | Affected               | Bypassed                       |
| Memory Usage                  | Shared memory          | Independent memory             |
| Communication                 | Shared variables       | Needs serialization (pickling) |
| Startup Overhead              | Low                    | Higher                         |
| Crash Isolation               | No                     | Yes                            |
| Parallelism                   | Simulated (concurrent) | True parallelism               |

---